Import all required libraries

In [8]:
from die import Die
from grid import grid
from numpy import array

Define the backtracking function

In [7]:
def backtrack(row: int,
              col: int,
              die: Die,
              score: int,
              turn: int,
              grid: array) -> bool:
    """
    Returns true if a satisfiable path has been found.
    """
    # Check: Have we veered off-grid?
    if row < 0 or row == len(grid):
        return False

    if col < 0 or col == len(grid[0]):
        return False

    # Check: Has this square been used already?
    c: int = grid[row][col]
    if c == 0:
        # There's no point going back to an old square
        # because that just means your score has not moved.
        return False

    # Check: Is there a number on this die-face?
    index_of_top_face: int = die.index_of_top_face()
    # Whether the top-face was set during this function call
    top_face_number_set_here: bool = False

    if not die.die_face_has_number(index_of_top_face):
        # Since c = s + t x n, n exists iff t divides c - s
        if (c - score) % turn == 0:
            die.set_die_face_number(index_of_top_face, (c - score) // turn)
            top_face_number_set_here = True
        else:
            # No integer will work as the die-face
            return False

    # Check: Given the number on the top-face, does the equation still hold?
    if c != score + turn * die.get_die_face_number(index_of_top_face):
        # We only enter this conditional if the die-face number was not set
        # in this function call, because if it was, n would have been chosen
        # by construction to satisfy the equation.
        #
        # Thus, we don't delete the die-face number here, but let the function-call
        # that was responsible for this die-face delete it.
        return False

    # Mark this square as visited
    grid[row][col] = 0

    # Check: Are we on the last square?
    if row == 0 and col == 5:
        return True

    # Check: Does rolling left work?
    die.tip_left()
    if backtrack(row, col - 1, die, c, turn + 1, grid):
        return True
    die.tip_right()

    # Check: Does rolling right work?
    die.tip_right()
    if backtrack(row, col + 1, die, c, turn + 1, grid):
        return True
    die.tip_left()
